# Run method for calculating p-values of SNPs combined over peaks

In [ ]:
setwd("../../InPlantaGWAS/11_Data_mining/QTLmineR/")

In [ ]:
source("ART.R")

library(data.table)

In [ ]:
file_path <- "shoot_4w_nothreshold_nodupfilter_keepoutliers_boxcox_outa3.assoc.txt"

## Prepare and inspect data

In [ ]:
result <-
  fread(file_path,
        header = TRUE)
colnames(result)[12] <- "P"

dir_snp_files <-
  "/media/helpdesk/Elements1/SLURMS_backup_6.3.21/oasis/naglemi/SKAT_testing/split_Chr_2to8parts/"

## Load functions

In [ ]:
find_peaks <- function(result, half_window_size, pvalue_threshold_peak,
                       pvalue_threshold_window, number_pvalue_threshold)
{
  windows <- list()

  peaks <- which(result$P < pvalue_threshold_peak)
  chr_peaks <- result$chr[peaks]

  # Only keep the peaks on chromesomes 1 to 19
  peaks <- peaks[chr_peaks %in% (1 : 19)]
  chr_peaks <- chr_peaks[chr_peaks %in% (1 : 19)]

  k <- 0
  for(chr in unique(chr_peaks))
  {
    peaks_chr <- peaks[chr_peaks == chr]
    for(i in 1 : length(peaks_chr))
    {
      ps_peak <- result$ps[peaks_chr[i]]
      ps_left <- ps_peak - half_window_size
      ps_right <- ps_peak + half_window_size

      window <- which(result$chr == chr & result$ps >= ps_left &
                        result$ps <= ps_right)
      #print(length(window))

      #print(sum(result[window, ]$p_wald < pvalue_threshold_window))
      if(sum(result[window, ]$P < pvalue_threshold_window,
             na.rm = TRUE) >=
         number_pvalue_threshold)
      {
        k <- k + 1
        #cat("k =", k, "\n")
        windows[[k]] <- result[window, ]
      }
    }
  }
    
  cat("max k =", k, "\n")
  return(windows)
}

In [ ]:
compute_window_pvalues <- function(windows, top_proportion = 0.5,
                                   decorrelation = FALSE,
                                   dir_snp_files = dir_snp_files)
{
  p_windows <- list()

  for(i in 1 : length(windows))
  {
    #print(i)
    window <- windows[[i]]
    d <- data.frame(rs = window$rs, chr = window$chr, pos = window$ps,
                    Z = window$beta/window$se)

    if(!decorrelation)
    {
      Z <- d$Z
      L <- length(Z)
      x <- Z^2
      k <- round(top_proportion * L)
      px <- 1 - pchisq(x, df = 1)
      P <- sort(px)
      P_rtp <- RTP(sum(-log(P[1 : k])), k, L)
      P_art <- ART(sum(log(P[1 : (k - 1)])), P[k], k, L)
      P_arta <- ART.A(P, k, L)
      #print(c(P_rtp, P_art, P_arta[1]))
      p_windows[[i]] <- c(P_art, P_rtp, P_arta[1])
    }
    else
    {
      chr <- window$chr[1]
      files_dir <- dir(dir_snp_files)
      snp_file_name_front <- paste("1323_cohort_mincount1_defaultmissingrates_Chr",
                                   chr, sep = "")
      nchar_snp_file_name_front <- nchar(snp_file_name_front)
      is_snp_file_name_front <-
        substr(files_dir, start = 1,
               stop = nchar_snp_file_name_front) == snp_file_name_front
      snp_files <- files_dir[is_snp_file_name_front]

      for(j in 1 : length(snp_files))
      {
        snp_file <- snp_files[j]
        #print(snp_file)
        gc()
        snp_data <- fread(paste(dir_snp_files, snp_file, sep = ""))
        gc()
        snp_data_pos_start <- snp_data$POS[1]
        snp_data_pos_end <- snp_data$POS[nrow(snp_data)]
        snp_window_pos_start <- window$ps[1]
        snp_window_pos_end <- window$ps[nrow(window)]

        if(snp_window_pos_start < snp_data_pos_start ||
           snp_window_pos_end > snp_data_pos_end) {
          #print("Wrong snp file!")
          break
        }
        else {
          snps_window <- subset(snp_data, SNP %in% window$rs)
          snps_window <- merge(window, snps_window, by.x = "rs", by.y = "SNP")

          #print(dim(snps_window))

          S <- cor(t(snps_window[, -(1 : 17)]), use = "complete.obs")

          # png(filename = paste("LD_window", i, ".png", sep = ""), width = 480*2, height = 480*2)
          # heatmap(S)
          # dev.off()

          ## the number of statistics
          Z <- d$Z
          L <- length(Z)
          ## eigen decomposition of the LD matrix
          ee <- eigen(S); eivec <- ee$vectors; eigva <- ee$values

          # eigva[eigva <= 1e-5] <- 1e-5

          pc <- eivec %*% diag(sqrt(1/eigva)) %*% t(eivec)
          # #print(dim(pc))
          ## calculate decorrelated statistics
          x <- (Z %*% pc)^2
          k <- round(top_proportion * L)
          px <- 1 - pchisq(x, df=1)
          P <- sort(px)
          P_rtp <- RTP(sum(-log(P[1 : k])), k, L)
          P_art <- ART(sum(log(P[1 : (k - 1)])), P[k], k, L)
          P_arta <- ART.A(P, k, L)
          #print(c(P_rtp, P_art, P_arta[1]))
          p_windows[[i]] <- c(P_art, P_rtp, P_arta[1])
        }
      }
    }
  }
  return(p_windows)
}

## Set parameters 

for finding findows

In [ ]:
half_window_size <- 500
pvalue_threshold_peak <- 1e-5
pvalue_threshold_window <- 1e-4
number_pvalue_threshold <- 5

for calculating window p-values

In [ ]:
top_proportion <- 0.5
decorrelation <- FALSE

In [ ]:
results_table <- fread("../../11_Data_mining/1_Table_results_by_method_a3_rmcrazygmmat.csv", na.strings = "")
#results_table <- fread("../11_Data_mining/1_Table_results_by_method_a3_rmcrazygmmat.csv", na.strings = "")

In [ ]:
outdir_base <- "Results/PeakWindowP/"

In [ ]:
library(tools)

In [ ]:
setwd("/mnt/data/NSF_GWAS/notebooks/InPlantaGWAS/11_Data_mining/")

In [ ]:
df <- data.frame()

In [ ]:
library(dplyr)

In [ ]:
dim(results_table)

In [ ]:
library(foreach)

In [ ]:
#results_combined <- foreach(i=1:nrow(results_table), .combine = "rbind")%do%{
for(h in c(1:nrow(results_table))){
    print(h)
    this_trait <- results_table[h, ]
    
    this_trait <- na.omit(unlist(this_trait))
    this_raw_trait_name <- this_trait['raw_trait']
    this_raw_trait_path <- this_trait['raw_trait_path']
    these_results_this_trait <- this_trait[3:length(this_trait)]

    trait_prefix <- basename(file_path_sans_ext(file_path_sans_ext(this_raw_trait_path)))

    output_dir <- paste0(outdir_base,
                         trait_prefix)
    
    if(!dir.exists(output_dir)) dir.create(output_dir, recursive = TRUE)
    
    if(length(na.omit(these_results_this_trait)) > 1){
    
        for(j in 1:length(these_results_this_trait)){

            if (grepl("gmmat", these_results_this_trait[j], ignore.case=TRUE)) method <- "GMMAT"
            if (grepl("gemma", these_results_this_trait[j], ignore.case=TRUE)) method <- "GEMMA"
            if (grepl("skat", these_results_this_trait[j], ignore.case=TRUE)) method <- "MTMCSKAT"
            if (grepl("farm", these_results_this_trait[j], ignore.case=TRUE)) method <- "FarmCPUpp"

            print(these_results_this_trait[j])
            print(method)
            
            batch <- names(these_results_this_trait[j])

            this_result_file <- these_results_this_trait[j]

            if(method == "GEMMA"){
                result <- fread(this_result_file)
                colnames(result)[12] <- "P"
                #print(head(result))
            }

            if(method == "GMMAT"){
                result <- fread(paste0(this_result_file, ".score.glmm"))
                colnames(result) <- c("rs", "chr", "ps",
                                      "ref", "alt", "n",
                                      "missrate", "af",
                                      "beta", "var", "P")
                result$se <- sqrt(result$var)
                #print(head(result))
            }

            if(method == "MTMCSKAT"){
                next
            }

            if(method == "FarmCPUpp"){
                next

            }

            windows <- try(find_peaks(result,
                                half_window_size,
                                pvalue_threshold_peak,
                                pvalue_threshold_window,
                                number_pvalue_threshold))

            if(length(windows) >= 1){
                pvalues_windows <- try(compute_window_pvalues(windows, top_proportion = top_proportion,
                                                              decorrelation = decorrelation,
                                                              dir_snp_files = dir_snp_files))

                window_summary <- foreach(i = 1:length(windows), .combine = "rbind") %do% {
                    window <- windows[[i]]
                    window_center_p <- min(window$P)

                    n_SNPs <- nrow(window)

                    window_center <- (window$ps[which(window$P == window_center_p)])[1]
                    window_center_chr <- as.character((window$chr[which(window$P == window_center_p)])[1])
                    # Take first element to handle cases where two adjacent have same p-val, but we need a 
                    #   specific position to merge by later (when merging with QTL Utils results)

                    window_pval <- pvalues_windows[[i]][3]

                    out <- c(window_center_chr, window_center, n_SNPs, window_center_p, window_pval)

                    out
                }

                if(length(windows) > 1){
                    window_summary <- as.data.table(window_summary)
                }

                if(length(windows) == 1){
                    window_summary.backup <- window_summary
                    window_summary <- as.data.table(t(window_summary))
                }

                colnames(window_summary) <- c("window_center_chr", "window_center", "n_SNPs", "window_center_p", "window_pval")

                window_summary$file_path <- this_raw_trait_path
                window_summary$batch <- batch
                window_summary$method <- method
                window_summary$raw_trait_name <- this_raw_trait_name
                



                df <- dplyr::bind_rows(df, window_summary)
                #return(window_summary)
            }
            


        } 
    } else {
        message(paste0("No results for trait ", this_raw_trait_name))

  }
}

In [ ]:
dim(df)

In [ ]:
head(df)

In [ ]:
levels(factor(df$raw_trait_name))

In [ ]:
length(windows)

In [ ]:
head(result)

In [ ]:
fwrite(df, "/mnt/data/NSF_GWAS/notebooks/InVitroRegenGWAS/06_Peak_pvals/3-OUT_Peak_P_a1_500bp_window.csv")

In [ ]:
Sys.time()

In [ ]:
getwd()

In [ ]:
head(df)